In [1]:
# necessary imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, mixed_precision
import matplotlib.pyplot as plt

In [2]:
# Enable mixed precision training to improve performance
# https://www.tensorflow.org/guide/mixed_precision
mixed_precision.set_global_policy('mixed_float16')

In [10]:
# Constants
AUTOTUNE = tf.data.AUTOTUNE # https://www.tensorflow.org/tutorials/images/transfer_learning?authuser=1#configure_the_dataset_for_performance
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
NUM_CLASSES = 4  # dry, wet, hazardous, e-waste
DATASET_DIR = "/kaggle/input/wastes/final_dataset_2"

In [11]:
# Data loading and preprocessing
def prepare_dataset(subset):
    ds = tf.keras.utils.image_dataset_from_directory(
        DATASET_DIR,
        seed=123,
        validation_split=0.2,
        subset=subset,
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE
    )
    ds = ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
    return ds

train_ds = prepare_dataset("training")
val_ds = prepare_dataset("validation")

Found 27091 files belonging to 4 classes.
Using 21673 files for training.
Found 27091 files belonging to 4 classes.
Using 5418 files for validation.


In [12]:
# Data augmentation
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
])

In [13]:
# Build model using Functional API
def build_model(num_classes):
    inputs = keras.Input(shape=IMG_SIZE + (3,))
    x = data_augmentation(inputs)

    # Use EfficientNetV2B0 as the base model
    base_model = keras.applications.EfficientNetV2B0(
        include_top=False, weights='imagenet', input_tensor=x
    )
    base_model.trainable = False

    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax', dtype='float32')(x)

    return keras.Model(inputs, outputs)

model = build_model(NUM_CLASSES)

24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [14]:
# Compile the model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [15]:
# Learning rate schedule
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=0.9
)

In [16]:
# Callbacks
callbacks = [
    keras.callbacks.ModelCheckpoint(
        'best_model.keras', save_best_only=True, monitor='val_accuracy'
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6
    ),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    keras.callbacks.TensorBoard(log_dir='./logs')
]

In [20]:
# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=callbacks
)

Epoch 1/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 40s 59ms/step - accuracy: 0.8989 - loss: 0.2716 - val_accuracy: 0.9474 - val_loss: 0.1571 - learning_rate: 0.0010
Epoch 2/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 40s 59ms/step - accuracy: 0.8994 - loss: 0.2742 - val_accuracy: 0.9456 - val_loss: 0.1538 - learning_rate: 0.0010
Epoch 3/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 40s 59ms/step - accuracy: 0.9058 - loss: 0.2590 - val_accuracy: 0.9413 - val_loss: 0.1582 - learning_rate: 0.0010
Epoch 4/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 40s 58ms/step - accuracy: 0.9041 - loss: 0.2621 - val_accuracy: 0.9461 - val_loss: 0.1497 - learning_rate: 0.0010
Epoch 5/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 40s 58ms/step - accuracy: 0.9106 - loss: 0.2478 - val_accuracy: 0.9430 - val_loss: 0.1525 - learning_rate: 0.0010
Epoch 6/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 40s 58ms/step - accuracy: 0.9093 - loss: 0.2549 - val_accuracy: 0.9443 - val_loss: 0.1531 - learning_rate: 0.0010
Epoch 7/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 39s 58ms/step - accuracy: 0.9087 - l

In [22]:
# Fine-tuning
model.trainable = True
for layer in model.layers[:-30]:
    layer.trainable = False

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=callbacks
)

Epoch 1/20


2024-09-21 11:26:03.184569: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_2_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


678/678 ━━━━━━━━━━━━━━━━━━━━ 61s 65ms/step - accuracy: 0.9141 - loss: 0.2384 - val_accuracy: 0.9492 - val_loss: 0.1404 - learning_rate: 1.0000e-05
Epoch 2/20
678/678 ━━━━━━━━━━━━━━━━━━━━ 40s 58ms/step - accuracy: 0.9145 - loss: 0.2369 - val_accuracy: 0.9491 - val_loss: 0.1414 - learning_rate: 1.0000e-05
Epoch 3/20
678/678 ━━━━━━━━━━━━━━━━━━━━ 40s 59ms/step - accuracy: 0.9143 - loss: 0.2359 - val_accuracy: 0.9481 - val_loss: 0.1422 - learning_rate: 1.0000e-05
Epoch 4/20
678/678 ━━━━━━━━━━━━━━━━━━━━ 39s 58ms/step - accuracy: 0.9123 - loss: 0.2371 - val_accuracy: 0.9494 - val_loss: 0.1410 - learning_rate: 1.0000e-05
Epoch 5/20
678/678 ━━━━━━━━━━━━━━━━━━━━ 39s 58ms/step - accuracy: 0.9111 - loss: 0.2356 - val_accuracy: 0.9480 - val_loss: 0.1427 - learning_rate: 1.0000e-05
Epoch 6/20
678/678 ━━━━━━━━━━━━━━━━━━━━ 39s 58ms/step - accuracy: 0.9119 - loss: 0.2359 - val_accuracy: 0.9472 - val_loss: 0.1442 - learning_rate: 1.0000e-05
Epoch 7/20
678/678 ━━━━━━━━━━━━━━━━━━━━ 39s 58ms/step - accurac

In [23]:
loss, accuracy = model.evaluate(val_ds)
print(f"Validation Accuracy: {accuracy:.4f}")

170/170 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - accuracy: 0.9502 - loss: 0.1413
Validation Accuracy: 0.9492


In [ ]:
# Evaluate the model
test_ds = prepare_dataset('test')
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test accuracy: {test_accuracy:.4f}")

# Function to plot training history
def plot_history(history):
    plt.figure(figsize=(12, 4))
    plt.subplot(121)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(122)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

plot_history(history)
plot_history(history_fine)

In [25]:
model.save('waste_classification_model.keras')

In [ ]:
def predict_waste_type(img_path):
    img = keras.utils.load_img(img_path, target_size=IMG_SIZE)
    img_array = keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)
    class_names = train_ds.class_names
    predicted_class = class_names[tf.argmax(predictions[0])]
    confidence = tf.reduce_max(predictions[0])

    return predicted_class, confidence

In [ ]:
# Example usage
img_path = 'path/to/your/image.jpg'
predicted_class, confidence = predict_waste_type(img_path)
print(f"Predicted class: {predicted_class}")
print(f"Confidence: {confidence:.4f}")